# Notebook to Practice Folium and Clustering

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import requests
from bs4 import BeautifulSoup
import time

print('Libraries imported.')

Libraries imported.


# Obtain data from zip-codes.com

In [2]:
# set url
base_url = 'https://www.zip-codes.com/county/in-lake.asp'
res = requests.get(base_url)
# print status code
res.status_code

200

In [3]:
# create soup from base url
soup = BeautifulSoup(res.content, 'lxml')

In [4]:
# examine soup object
# print(soup.prettify())

In [5]:
# create table 
table = soup.find('table',{'class':'statTable'})

In [6]:
# examine table
# print(table.prettify())

In [7]:
# table.find_all('td',{'class':'label'})

In [8]:
# cell used to isolate info
for i in table.find_all('td',{'class':'label'}):
    try:
        print(i.find('a').attrs['title'])
    except:
        None

ZIP Code 46303, CEDAR LAKE, IN
ZIP Code 46307, CROWN POINT, IN
ZIP Code 46308, CROWN POINT, IN
ZIP Code 46311, DYER, IN
ZIP Code 46312, EAST CHICAGO, IN
ZIP Code 46319, GRIFFITH, IN
ZIP Code 46320, HAMMOND, IN
ZIP Code 46321, MUNSTER, IN
ZIP Code 46322, HIGHLAND, IN
ZIP Code 46323, HAMMOND, IN
ZIP Code 46324, HAMMOND, IN
ZIP Code 46325, HAMMOND, IN
ZIP Code 46327, HAMMOND, IN
ZIP Code 46342, HOBART, IN
ZIP Code 46355, LEROY, IN
ZIP Code 46356, LOWELL, IN
ZIP Code 46373, SAINT JOHN, IN
ZIP Code 46375, SCHERERVILLE, IN
ZIP Code 46376, SCHNEIDER, IN
ZIP Code 46377, SHELBY, IN
ZIP Code 46394, WHITING, IN
ZIP Code 46401, GARY, IN
ZIP Code 46402, GARY, IN
ZIP Code 46403, GARY, IN
ZIP Code 46404, GARY, IN
ZIP Code 46405, LAKE STATION, IN
ZIP Code 46406, GARY, IN
ZIP Code 46407, GARY, IN
ZIP Code 46408, GARY, IN
ZIP Code 46409, GARY, IN
ZIP Code 46410, MERRILLVILLE, IN
ZIP Code 46411, MERRILLVILLE, IN


In [9]:
# loop through table to obtain desired info and create dataframe
cities = []

for i in table.find_all('td',{'class':'label'}):
    try:
        city = {}
    
        city['Name'] = i.find('a').attrs['title'].replace('ZIP Code ', '').replace(', IN', '')
      
        cities.append(city)
    except:
        None

cities_df = pd.DataFrame(cities)
cities_df

Name
0     46303, CEDAR LAKE
1    46307, CROWN POINT
2    46308, CROWN POINT
3           46311, DYER
4   46312, EAST CHICAGO
5       46319, GRIFFITH
6        46320, HAMMOND
7        46321, MUNSTER
8       46322, HIGHLAND
9        46323, HAMMOND
10       46324, HAMMOND
11       46325, HAMMOND
12       46327, HAMMOND
13        46342, HOBART
14         46355, LEROY
15        46356, LOWELL
16    46373, SAINT JOHN
17  46375, SCHERERVILLE
18     46376, SCHNEIDER
19        46377, SHELBY
20       46394, WHITING
21          46401, GARY
22          46402, GARY
23          46403, GARY
24          46404, GARY
25  46405, LAKE STATION
26          46406, GARY
27          46407, GARY
28          46408, GARY
29          46409, GARY
30  46410, MERRILLVILLE
31  46411, MERRILLVILLE

In [10]:
for i in cities_df['Name']:
    print(i.split(sep=', ')[0])

46303
46307
46308
46311
46312
46319
46320
46321
46322
46323
46324
46325
46327
46342
46355
46356
46373
46375
46376
46377
46394
46401
46402
46403
46404
46405
46406
46407
46408
46409
46410
46411


In [11]:
news = []

for i in cities_df['Name']:
    new = {}
    new['neighborhood'] = i
    new['postalcode'] = i.split(sep=', ')[0]
    new['city'] = i.split(sep=', ',)[1]
    
    news.append(new)
    
lake_county = pd.DataFrame(news)
lake_county

city         neighborhood postalcode
0     CEDAR LAKE    46303, CEDAR LAKE      46303
1    CROWN POINT   46307, CROWN POINT      46307
2    CROWN POINT   46308, CROWN POINT      46308
3           DYER          46311, DYER      46311
4   EAST CHICAGO  46312, EAST CHICAGO      46312
5       GRIFFITH      46319, GRIFFITH      46319
6        HAMMOND       46320, HAMMOND      46320
7        MUNSTER       46321, MUNSTER      46321
8       HIGHLAND      46322, HIGHLAND      46322
9        HAMMOND       46323, HAMMOND      46323
10       HAMMOND       46324, HAMMOND      46324
11       HAMMOND       46325, HAMMOND      46325
12       HAMMOND       46327, HAMMOND      46327
13        HOBART        46342, HOBART      46342
14         LEROY         46355, LEROY      46355
15        LOWELL        46356, LOWELL      46356
16    SAINT JOHN    46373, SAINT JOHN      46373
17  SCHERERVILLE  46375, SCHERERVILLE      46375
18     SCHNEIDER     46376, SCHNEIDER      46376
19        SHELBY        46377, SHELBY      46377
20       WHITING       46394, WHITING      46394
21          GARY          46401, GARY      46401
22          GARY          46402, GARY      46402
23          GARY          46403, GARY      46403
24          GARY          46404, GARY      46404
25  LAKE STATION  46405, LAKE STATION      46405
26          GARY          46406, GARY      46406
27          GARY          46407, GARY      46407
28          GARY          46408, GARY      46408
29          GARY          46409, GARY      46409
30  MERRILLVILLE  46410, MERRILLVILLE      46410
31  MERRILLVILLE  46411, MERRILLVILLE      46411

In [12]:
lake_county.shape

(32, 3)

In [13]:
address = 'Lake County, IN'

geolocator = Nominatim(user_agent="ec_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lake County, IN are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lake County, IN are 41.4193829, -87.3692227.


In [14]:
geolocator.geocode('East Chicago', '46312')

Location(East Chicago, Lake County, Indiana, 46312, United States of America, (41.6397857, -87.4548466, 0.0))

In [15]:
lake_county['neighborhood'] = lake_county['city'] + ', ' + lake_county['postalcode']
lake_county.head()

city         neighborhood postalcode
0    CEDAR LAKE    CEDAR LAKE, 46303      46303
1   CROWN POINT   CROWN POINT, 46307      46307
2   CROWN POINT   CROWN POINT, 46308      46308
3          DYER          DYER, 46311      46311
4  EAST CHICAGO  EAST CHICAGO, 46312      46312

In [16]:
# get lat and long coordinates for each zip code in lake_county dataframe
geolocator = Nominatim(user_agent="ec_explorer")

coords = []

for row in lake_county['neighborhood']:
    coord = {}

    location = geolocator.geocode(row)
    
    coord['Latitude'] =  location.latitude
    coord['Longitude'] = location.longitude
    
    print(row)
    
    coords.append(coord)

coords_df = pd.DataFrame(coords)

CEDAR LAKE, 46303
CROWN POINT, 46307
CROWN POINT, 46308
DYER, 46311
EAST CHICAGO, 46312
GRIFFITH, 46319
HAMMOND, 46320
MUNSTER, 46321
HIGHLAND, 46322
HAMMOND, 46323
HAMMOND, 46324
HAMMOND, 46325
HAMMOND, 46327
HOBART, 46342
LEROY, 46355
LOWELL, 46356
SAINT JOHN, 46373
SCHERERVILLE, 46375
SCHNEIDER, 46376
SHELBY, 46377
WHITING, 46394
GARY, 46401
GARY, 46402
GARY, 46403
GARY, 46404
LAKE STATION, 46405
GARY, 46406
GARY, 46407
GARY, 46408
GARY, 46409
MERRILLVILLE, 46410
MERRILLVILLE, 46411


In [17]:
df = pd.concat([lake_county, coords_df], axis=1)
df

city         neighborhood postalcode   Latitude  Longitude
0     CEDAR LAKE    CEDAR LAKE, 46303      46303  41.364758 -87.441147
1    CROWN POINT   CROWN POINT, 46307      46307  41.416981 -87.365314
2    CROWN POINT   CROWN POINT, 46308      46308  30.665408 -88.206603
3           DYER          DYER, 46311      46311  41.494202 -87.521707
4   EAST CHICAGO  EAST CHICAGO, 46312      46312  41.639786 -87.454847
5       GRIFFITH      GRIFFITH, 46319      46319  41.534507 -87.425530
6        HAMMOND       HAMMOND, 46320      46320  41.583366 -87.500043
7        MUNSTER       MUNSTER, 46321      46321  41.564480 -87.512541
8       HIGHLAND      HIGHLAND, 46322      46322  41.553647 -87.451984
9        HAMMOND       HAMMOND, 46323      46323  41.583366 -87.500043
10       HAMMOND       HAMMOND, 46324      46324  41.583366 -87.500043
11       HAMMOND       HAMMOND, 46325      46325  41.583366 -87.500043
12       HAMMOND       HAMMOND, 46327      46327  41.583366 -87.500043
13        HOBART        HOBART, 46342      46342  41.532259 -87.255035
14         LEROY         LEROY, 46355      46355  41.360037 -87.271978
15        LOWELL        LOWELL, 46356      46356  41.291424 -87.420590
16    SAINT JOHN    SAINT JOHN, 46373      46373  41.450036 -87.470038
17  SCHERERVILLE  SCHERERVILLE, 46375      46375  41.478925 -87.454761
18     SCHNEIDER     SCHNEIDER, 46376      46376  41.186979 -87.448367
19        SHELBY        SHELBY, 46377      46377  41.195313 -87.347810
20       WHITING       WHITING, 46394      46394  41.679758 -87.494487
21          GARY          GARY, 46401      46401  41.602129 -87.337137
22          GARY          GARY, 46402      46402  41.602129 -87.337137
23          GARY          GARY, 46403      46403  41.602129 -87.337137
24          GARY          GARY, 46404      46404  41.602129 -87.337137
25  LAKE STATION  LAKE STATION, 46405      46405  41.575037 -87.238925
26          GARY          GARY, 46406      46406  41.602129 -87.337137
27          GARY          GARY, 46407      46407  41.602129 -87.337137
28          GARY          GARY, 46408      46408  41.602129 -87.337137
29          GARY          GARY, 46409      46409  41.602129 -87.337137
30  MERRILLVILLE  MERRILLVILLE, 46410      46410  41.482814 -87.332814
31  MERRILLVILLE  MERRILLVILLE, 46411      46411  41.482814 -87.332814

In [18]:
df.dtypes

city             object
neighborhood     object
postalcode       object
Latitude        float64
Longitude       float64
dtype: object

In [19]:
address = 'Lake County, IN'

geolocator = Nominatim(user_agent="ec_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lake County, IN are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lake County, IN are 41.4193829, -87.3692227.


In [20]:
# create map of Lake County, IN using latitude and longitude values
map_lake_county = folium.Map(location=[latitude, longitude], zoom_start=9)
map_lake_county

In [21]:
# create map of LA using latitude and longitude values
map_lake_county = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lake_county)  
    
map_lake_county

In [22]:
address = 'Lake County, IN'

geolocator = Nominatim(user_agent="ec_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lake County, IN are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lake County, IN are 41.4193829, -87.3692227.


In [23]:
neighborhood_latitude = latitude
neighborhood_longitude = longitude

In [25]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [26]:
# url # display URL

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood', 
                  'neighborhood Latitude', 
                  'neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
lake_county_venues = getNearbyVenues(names=df['neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

CEDAR LAKE, 46303
CROWN POINT, 46307
CROWN POINT, 46308
DYER, 46311
EAST CHICAGO, 46312
GRIFFITH, 46319
HAMMOND, 46320
MUNSTER, 46321
HIGHLAND, 46322
HAMMOND, 46323
HAMMOND, 46324
HAMMOND, 46325
HAMMOND, 46327
HOBART, 46342
LEROY, 46355
LOWELL, 46356
SAINT JOHN, 46373
SCHERERVILLE, 46375
SCHNEIDER, 46376
SHELBY, 46377
WHITING, 46394
GARY, 46401
GARY, 46402
GARY, 46403
GARY, 46404
LAKE STATION, 46405
GARY, 46406
GARY, 46407
GARY, 46408
GARY, 46409
MERRILLVILLE, 46410
MERRILLVILLE, 46411


In [29]:
# lake_county_venues

In [30]:
lake_county_venues.shape

(269, 7)

# Plot Lake County Venues

In [31]:
# create map of Lake county venues using latitude and longitude values
map_lake_county = folium.Map(location=[latitude, longitude], zoom_start=9)


# instantiate a feature group for the incidents in the dataframe
venues = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, name in zip(lake_county_venues['Venue Latitude'], lake_county_venues['Venue Longitude'], lake_county_venues['Venue']):
    venues.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            popup=name,
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )


map_lake_county.add_child(venues)

In [58]:
import plotly.graph_objects as go
import plotly.io as pio

df1 = lake_county_venues[['neighborhood', 'Venue', 'Venue Category']]


fig = go.Figure(data=[go.Table(
    header=dict(values=list(df1.columns),
                line_color='darkslategray',
                fill_color='#00BFFF',
                align='center',
                font=dict(color='white', size=18)),
    cells=dict(values=[df1['neighborhood'], df1['Venue'], df1['Venue Category']],
               fill_color='#F0FFFF',
               line_color='darkslategray',
               align='center',
               font=dict(color='black', size=14))
)])

fig.show()

In [59]:
pio.write_html(fig, file='venues.html', auto_open=False)

In [34]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
map_lake_county = folium.Map(location = [latitude, longitude], zoom_start = 9.5)

# instantiate a mark cluster object for the incidents in the dataframe
venues = plugins.MarkerCluster().add_to(map_lake_county)

# add link to graphs
popup = folium.Popup('<a href="https://mariojrgithub.github.io/ca_wildfires/graphs" target="_blank">Venue Table</a>')

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(lake_county_venues['Venue Latitude'], lake_county_venues['Venue Longitude'], lake_county_venues['Venue']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(venues)


# add dateframe to map    
folium.map.Marker(
    [latitude, longitude],
    icon=folium.Icon(), 
    popup=popup
).add_to(map_lake_county)    
    
# display map
map_lake_county

In [35]:
# save as html file
map_lake_county.save('index.html')

# Group Venues by Zip

In [36]:
lake_county_venues.groupby('neighborhood').count()

neighborhood Latitude  neighborhood Longitude  Venue  \
neighborhood                                                                
CEDAR LAKE, 46303                        2                       2      2   
CROWN POINT, 46307                      27                      27     27   
CROWN POINT, 46308                       4                       4      4   
DYER, 46311                             21                      21     21   
EAST CHICAGO, 46312                      9                       9      9   
GARY, 46401                              7                       7      7   
GARY, 46402                              7                       7      7   
GARY, 46403                              7                       7      7   
GARY, 46404                              7                       7      7   
GARY, 46406                              7                       7      7   
GARY, 46407                              7                       7      7   
GARY, 46408                              7                       7      7   
GARY, 46409                              7                       7      7   
GRIFFITH, 46319                          8                       8      8   
HAMMOND, 46320                           3                       3      3   
HAMMOND, 46323                           3                       3      3   
HAMMOND, 46324                           3                       3      3   
HAMMOND, 46325                           3                       3      3   
HAMMOND, 46327                           3                       3      3   
HIGHLAND, 46322                          8                       8      8   
HOBART, 46342                           10                      10     10   
LAKE STATION, 46405                     16                      16     16   
LEROY, 46355                             1                       1      1   
LOWELL, 46356                           12                      12     12   
MERRILLVILLE, 46410                      4                       4      4   
MERRILLVILLE, 46411                      4                       4      4   
MUNSTER, 46321                          37                      37     37   
SAINT JOHN, 46373                       13                      13     13   
SCHERERVILLE, 46375                      1                       1      1   
SCHNEIDER, 46376                         1                       1      1   
SHELBY, 46377                            2                       2      2   
WHITING, 46394                          18                      18     18   

                     Venue Latitude  Venue Longitude  Venue Category  
neighborhood                                                          
CEDAR LAKE, 46303                 2                2               2  
CROWN POINT, 46307               27               27              27  
CROWN POINT, 46308                4                4               4  
DYER, 46311                      21               21              21  
EAST CHICAGO, 46312               9                9               9  
GARY, 46401                       7                7               7  
GARY, 46402                       7                7               7  
GARY, 46403                       7                7               7  
GARY, 46404                       7                7               7  
GARY, 46406                       7                7               7  
GARY, 46407                       7                7               7  
GARY, 46408                       7                7               7  
GARY, 46409                       7                7               7  
GRIFFITH, 46319                   8                8               8  
HAMMOND, 46320                    3                3               3  
HAMMOND, 46323                    3                3               3  
HAMMOND, 46324                    3                3               3  
HAMMOND, 46325                    3                3 

In [37]:
print('There are {} uniques categories.'.format(len(lake_county_venues['Venue Category'].unique())))

There are 78 uniques categories.


In [38]:
# one hot encoding
lake_county_onehot = pd.get_dummies(lake_county_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lake_county_onehot['neighborhood'] = lake_county_venues['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lake_county_onehot.columns[-1]] + list(lake_county_onehot.columns[:-1])
lake_county_onehot = lake_county_onehot[fixed_columns]

lake_county_onehot.head()

neighborhood  American Restaurant  BBQ Joint  Bakery  Bank  Bar  \
0   CEDAR LAKE, 46303                    0          0       0     0    0   
1   CEDAR LAKE, 46303                    0          0       0     0    0   
2  CROWN POINT, 46307                    0          0       0     0    0   
3  CROWN POINT, 46307                    0          0       0     0    0   
4  CROWN POINT, 46307                    0          0       0     0    0   

   Baseball Field  Baseball Stadium  Bistro  Border Crossing  Bowling Alley  \
0               0                 0       0                0              0   
1               0                 0       0                0              0   
2               0                 0       0                0              0   
3               0                 0       0                0              0   
4               0                 0       0                0              0   

   Breakfast Spot  Brewery  Burger Joint  Bus Station  Business Service  \
0               0        0             0            0                 0   
1               0        0             0            0                 0   
2               0        0             0            0                 0   
3               0        0             0            0                 0   
4               0        1             0            0                 0   

   Butcher  Café  Carpet Store  Chinese Restaurant  Comfort Food Restaurant  \
0        0     0             0                   0                        0   
1        0     0             0                   0                        0   
2        0     0             0                   0                        0   
3        0     0             0                   0                        0   
4        0     0             0                   0                        0   

   Comic Shop  Construction & Landscaping  Convenience Store  Credit Union  \
0           0                           0                  0             0   
1           0                           0                  0             0   
2           1                           0                  0             0   
3           0                           0                  0             0   
4           0                           0                  0             0   

   Currency Exchange  Deli / Bodega  Diner  Discount Store  Doctor's Office  \
0                  0              0      0               0                0   
1                  0              0      0               0                0   
2                  0              0      0               0                0   
3                  0              0      0               0                0   
4                  0              0      0               0                0   

   Donut Shop  Farmers Market  Fast Food Restaurant  \
0           0               0                     0   
1           0               0                     0   
2           0               0                     0   
3           0               0                     0   
4           0               0                     0   

   Financial or Legal Service  Food  Fried Chicken Joint  Frozen Yogurt Shop  \
0                           0     0                    0                   0   
1                           0     0                    0                   0   
2                           0     0                    0                   0   
3                           0     0                    0                   0   
4                           0     0                    0                   0   

   Furniture / Home Store  Gas Station  General Entertainment  \
0                       0            0                      0   
1                       0            0                      0   
2                       0            0                      0   
3                       0            0                      0   
4                       0            0                      0   

   G

In [39]:
lake_county_onehot.shape

(269, 79)

In [40]:
lake_county_grouped = lake_county_onehot.groupby('neighborhood').mean().reset_index()
lake_county_grouped.head()

neighborhood  American Restaurant  BBQ Joint  Bakery      Bank  \
0    CEDAR LAKE, 46303             0.000000        0.0     0.0  0.000000   
1   CROWN POINT, 46307             0.074074        0.0     0.0  0.111111   
2   CROWN POINT, 46308             0.000000        0.0     0.0  0.000000   
3          DYER, 46311             0.095238        0.0     0.0  0.000000   
4  EAST CHICAGO, 46312             0.000000        0.0     0.0  0.000000   

        Bar  Baseball Field  Baseball Stadium  Bistro  Border Crossing  \
0  0.000000             0.0               0.0     0.0         0.000000   
1  0.222222             0.0               0.0     0.0         0.000000   
2  0.000000             0.0               0.0     0.0         0.000000   
3  0.047619             0.0               0.0     0.0         0.047619   
4  0.000000             0.0               0.0     0.0         0.000000   

   Bowling Alley  Breakfast Spot   Brewery  Burger Joint  Bus Station  \
0       0.000000             0.0  0.000000           0.0          0.0   
1       0.037037             0.0  0.037037           0.0          0.0   
2       0.000000             0.0  0.000000           0.0          0.0   
3       0.000000             0.0  0.047619           0.0          0.0   
4       0.000000             0.0  0.000000           0.0          0.0   

   Business Service  Butcher      Café  Carpet Store  Chinese Restaurant  \
0               0.0      0.0  0.000000           0.0                 0.0   
1               0.0      0.0  0.148148           0.0                 0.0   
2               0.0      0.0  0.000000           0.0                 0.0   
3               0.0      0.0  0.047619           0.0                 0.0   
4               0.0      0.0  0.000000           0.0                 0.0   

   Comfort Food Restaurant  Comic Shop  Construction & Landscaping  \
0                      0.0    0.000000                         0.0   
1                      0.0    0.037037                         0.0   
2                      0.0    0.000000                         0.0   
3                      0.0    0.000000                         0.0   
4                      0.0    0.000000                         0.0   

   Convenience Store  Credit Union  Currency Exchange  Deli / Bodega  Diner  \
0                0.0           0.0           0.000000            0.0    0.0   
1                0.0           0.0           0.000000            0.0    0.0   
2                0.0           0.0           0.000000            0.0    0.0   
3                0.0           0.0           0.000000            0.0    0.0   
4                0.0           0.0           0.111111            0.0    0.0   

   Discount Store  Doctor's Office  Donut Shop  Farmers Market  \
0        0.000000              0.0    0.000000        0.000000   
1        0.000000              0.0    0.000000        0.000000   
2        0.000000              0.0    0.000000        0.000000   
3        0.047619              0.0    0.047619        0.047619   
4        0.111111              0.0    0.111111        0.000000   

   Fast Food Restaurant  Financial or Legal Service      Food  \
0              0.000000                         0.0  0.000000   
1              0.037037                         0.0  0.000000   
2              0.000000                         0.0  0.000000   
3              0.000000                         0.0  0.047619   
4              0.222222                         0.0  0.000000   

   Fried Chicken Joint  Frozen Yogurt Shop  Furniture / Home Store  \
0                  0.0            0.000000                0.000000   
1                  0.0            0.000000                0.000000   
2                  0.0            0.000000                0.000000   
3                  0.0            0.047619                0.095238   
4                  0.0            0.000000                0.000000   

   Gas Station  General Entertainment  Greek Restaurant  Grocery Store  Gym  \
0     0.000000              

In [41]:
# num_top_venues = 5

# for hood in lake_county_grouped['neighborhood']:
#     print("----"+hood+"----")
#     temp = lake_county_grouped[lake_county_grouped['neighborhood'] == hood].T.reset_index()
#     temp.columns = ['venue','freq']
#     temp = temp.iloc[1:]
#     temp['freq'] = temp['freq'].astype(float)
#     temp = temp.round({'freq': 2})
#     print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#     print('\n')

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
lake_county_venues_sorted = pd.DataFrame(columns=columns)
lake_county_venues_sorted['neighborhood'] = lake_county_grouped['neighborhood']

for ind in np.arange(lake_county_grouped.shape[0]):
    lake_county_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lake_county_grouped.iloc[ind, :], num_top_venues)

lake_county_venues_sorted.head()

neighborhood 1st Most Common Venue   2nd Most Common Venue  \
0    CEDAR LAKE, 46303          Home Service             Video Store   
1   CROWN POINT, 46307                   Bar                    Café   
2   CROWN POINT, 46308                  Lake    Gym / Fitness Center   
3          DYER, 46311   American Restaurant  Furniture / Home Store   
4  EAST CHICAGO, 46312        Sandwich Place    Fast Food Restaurant   

  3rd Most Common Venue       4th Most Common Venue 5th Most Common Venue  \
0       Doctor's Office  Construction & Landscaping     Convenience Store   
1                  Bank         American Restaurant           Pizza Place   
2          Intersection           Health Food Store           Video Store   
3          Liquor Store                 Pizza Place               Brewery   
4    Mexican Restaurant                  Donut Shop    Seafood Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0          Credit Union     Currency Exchange         Deli / Bodega   
1                   Pub               Brewery                 Plaza   
2        Discount Store     Convenience Store          Credit Union   
3           Gas Station                  Food        Farmers Market   
4     Currency Exchange        Discount Store         Deli / Bodega   

  9th Most Common Venue      10th Most Common Venue  
0                 Diner              Discount Store  
1            Comic Shop               Bowling Alley  
2     Currency Exchange               Deli / Bodega  
3            Donut Shop              Discount Store  
4            Comic Shop  Construction & Landscaping

# Map Clusters

In [44]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [45]:
# set number of clusters
kclusters = 5

lake_county_grouped_clustering = lake_county_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=0, n_init=12).fit(lake_county_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 1, 3, 3, 3, 3, 3])

In [46]:
# add clustering labels
lake_county_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

lake_county_merged = df

# merge lake_county_grouped with lake_county data to add latitude/longitude for each neighborhood
lake_county_merged = lake_county_merged.join(lake_county_venues_sorted.set_index('neighborhood'), on='neighborhood')
lake_county_merged.head() 

city         neighborhood postalcode   Latitude  Longitude  \
0    CEDAR LAKE    CEDAR LAKE, 46303      46303  41.364758 -87.441147   
1   CROWN POINT   CROWN POINT, 46307      46307  41.416981 -87.365314   
2   CROWN POINT   CROWN POINT, 46308      46308  30.665408 -88.206603   
3          DYER          DYER, 46311      46311  41.494202 -87.521707   
4  EAST CHICAGO  EAST CHICAGO, 46312      46312  41.639786 -87.454847   

   Cluster Labels 1st Most Common Venue   2nd Most Common Venue  \
0               0          Home Service             Video Store   
1               1                   Bar                    Café   
2               1                  Lake    Gym / Fitness Center   
3               1   American Restaurant  Furniture / Home Store   
4               1        Sandwich Place    Fast Food Restaurant   

  3rd Most Common Venue       4th Most Common Venue 5th Most Common Venue  \
0       Doctor's Office  Construction & Landscaping     Convenience Store   
1                  Bank         American Restaurant           Pizza Place   
2          Intersection           Health Food Store           Video Store   
3          Liquor Store                 Pizza Place               Brewery   
4    Mexican Restaurant                  Donut Shop    Seafood Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0          Credit Union     Currency Exchange         Deli / Bodega   
1                   Pub               Brewery                 Plaza   
2        Discount Store     Convenience Store          Credit Union   
3           Gas Station                  Food        Farmers Market   
4     Currency Exchange        Discount Store         Deli / Bodega   

  9th Most Common Venue      10th Most Common Venue  
0                 Diner              Discount Store  
1            Comic Shop               Bowling Alley  
2     Currency Exchange               Deli / Bodega  
3            Donut Shop              Discount Store  
4            Comic Shop  Construction & Landscaping

In [47]:
lake_county_merged.isnull().sum()

city                      0
neighborhood              0
postalcode                0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

In [48]:
lake_county_merged.dtypes

city                       object
neighborhood               object
postalcode                 object
Latitude                  float64
Longitude                 float64
Cluster Labels              int32
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [49]:
lake_county_merged.shape

(32, 16)

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lake_county_merged['Latitude'], lake_county_merged['Longitude'], lake_county_merged['neighborhood'], lake_county_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [51]:
lake_county_merged.loc[lake_county_merged['Cluster Labels'] == 0, 
                     lake_county_merged.columns[[0,2] + list(range(5, lake_county_merged.shape[1]))]]

city postalcode  Cluster Labels 1st Most Common Venue  \
0  CEDAR LAKE      46303               0          Home Service   

  2nd Most Common Venue 3rd Most Common Venue       4th Most Common Venue  \
0           Video Store       Doctor's Office  Construction & Landscaping   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0     Convenience Store          Credit Union     Currency Exchange   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0         Deli / Bodega                 Diner         Discount Store

In [52]:
lake_county_merged.loc[lake_county_merged['Cluster Labels'] == 1, 
                     lake_county_merged.columns[[0,2] + list(range(5, lake_county_merged.shape[1]))]]

city postalcode  Cluster Labels 1st Most Common Venue  \
1    CROWN POINT      46307               1                   Bar   
2    CROWN POINT      46308               1                  Lake   
3           DYER      46311               1   American Restaurant   
4   EAST CHICAGO      46312               1        Sandwich Place   
5       GRIFFITH      46319               1        Baseball Field   
7        MUNSTER      46321               1    Mexican Restaurant   
8       HIGHLAND      46322               1     Polish Restaurant   
13        HOBART      46342               1    Mexican Restaurant   
15        LOWELL      46356               1    Mexican Restaurant   
16    SAINT JOHN      46373               1                 Diner   
18     SCHNEIDER      46376               1                  Park   
19        SHELBY      46377               1          Squash Court   
20       WHITING      46394               1                  Café   
25  LAKE STATION      46405               1  Fast Food Restaurant   
30  MERRILLVILLE      46410               1        Breakfast Spot   
31  MERRILLVILLE      46411               1        Breakfast Spot   

     2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
1                     Café                  Bank   American Restaurant   
2     Gym / Fitness Center          Intersection     Health Food Store   
3   Furniture / Home Store          Liquor Store           Pizza Place   
4     Fast Food Restaurant    Mexican Restaurant            Donut Shop   
5    General Entertainment    Chinese Restaurant         Grocery Store   
7                      Bar   American Restaurant            Smoke Shop   
8        Convenience Store                Office        Baseball Field   
13          Breakfast Spot         Movie Theater                  Bank   
15    Gym / Fitness Center           Pizza Place               Brewery   
16         Doctor's Office    Mexican Restaurant            Nail Salon   
18             Video Store    Chinese Restaurant            Comic Shop   
19                     Bar           Video Store       Doctor's Office   
20                Pharmacy           Pizza Place                   Bar   
25            Liquor Store            Donut Shop              Pharmacy   
30            Carpet Store                 Diner           Video Store   
31            Carpet Store                 Diner           Video Store   

         5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
1                  Pizza Place                   Pub               Brewery   
2                  Video Store        Discount Store     Convenience Store   
3                      Brewery           Gas Station                  Food   
4           Seafood Restaurant     Currency Exchange        Discount Store   
5            Convenience Store          Credit Union          Intersection   
7               Ice Cream Shop        Sandwich Place  Fast Food Restaurant   
8                         Park   Peruvian Restaurant              Pharmacy   
13                         Bar  Fast Food Restaurant                Bistro   
15     Health & Beauty Service            Restaurant        Discount Store   
16                    Pharmacy        Breakfast Spot    Chinese Restaurant   
18  Construction & Landscaping     Convenience Store          Credit Union   
19  Construction & Landscaping     Convenience Store          Credit Union   
20        Fast Food Restaurant               Brewery        Discount Store   
25                 Pizza Place         Hot Dog Joint        Sandwich Place   
30                  Comic Shop     Convenience Store          Credit Union   
31                  Comic Shop     Convenience Store          Credit Union   

         8th Most Common Venue 9th Most Common Venue  \
1                        Plaza            Comic Shop   
2                 Credit Union     Currency Exchange   
3               Farmers Market            Donut Shop   
4                Deli / Bode

In [53]:
lake_county_merged.loc[lake_county_merged['Cluster Labels'] == 2, 
                     lake_county_merged.columns[[0,2] + list(range(5, lake_county_merged.shape[1]))]]

city postalcode  Cluster Labels 1st Most Common Venue  \
6        HAMMOND      46320               2           Video Store   
9        HAMMOND      46323               2           Video Store   
10       HAMMOND      46324               2           Video Store   
11       HAMMOND      46325               2           Video Store   
12       HAMMOND      46327               2           Video Store   
17  SCHERERVILLE      46375               2      Business Service   

      2nd Most Common Venue    3rd Most Common Venue  \
6   Comfort Food Restaurant         Business Service   
9   Comfort Food Restaurant         Business Service   
10  Comfort Food Restaurant         Business Service   
11  Comfort Food Restaurant         Business Service   
12  Comfort Food Restaurant         Business Service   
17              Video Store  Comfort Food Restaurant   

         4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
6   Construction & Landscaping     Convenience Store          Credit Union   
9   Construction & Landscaping     Convenience Store          Credit Union   
10  Construction & Landscaping     Convenience Store          Credit Union   
11  Construction & Landscaping     Convenience Store          Credit Union   
12  Construction & Landscaping     Convenience Store          Credit Union   
17  Construction & Landscaping     Convenience Store          Credit Union   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
6      Currency Exchange         Deli / Bodega                 Diner   
9      Currency Exchange         Deli / Bodega                 Diner   
10     Currency Exchange         Deli / Bodega                 Diner   
11     Currency Exchange         Deli / Bodega                 Diner   
12     Currency Exchange         Deli / Bodega                 Diner   
17     Currency Exchange         Deli / Bodega                 Diner   

   10th Most Common Venue  
6          Discount Store  
9          Discount Store  
10         Discount Store  
11         Discount Store  
12         Discount Store  
17         Discount Store

In [54]:
lake_county_merged.loc[lake_county_merged['Cluster Labels'] == 3, 
                     lake_county_merged.columns[[0,2] + list(range(5, lake_county_merged.shape[1]))]]

city postalcode  Cluster Labels 1st Most Common Venue  \
21  GARY      46401               3   American Restaurant   
22  GARY      46402               3   American Restaurant   
23  GARY      46403               3   American Restaurant   
24  GARY      46404               3   American Restaurant   
26  GARY      46406               3   American Restaurant   
27  GARY      46407               3   American Restaurant   
28  GARY      46408               3   American Restaurant   
29  GARY      46409               3   American Restaurant   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
21      Baseball Stadium                   Gym                  Food   
22      Baseball Stadium                   Gym                  Food   
23      Baseball Stadium                   Gym                  Food   
24      Baseball Stadium                   Gym                  Food   
26      Baseball Stadium                   Gym                  Food   
27      Baseball Stadium                   Gym                  Food   
28      Baseball Stadium                   Gym                  Food   
29      Baseball Stadium                   Gym                  Food   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
21           Bus Station    Chinese Restaurant                  Bank   
22           Bus Station    Chinese Restaurant                  Bank   
23           Bus Station    Chinese Restaurant                  Bank   
24           Bus Station    Chinese Restaurant                  Bank   
26           Bus Station    Chinese Restaurant                  Bank   
27           Bus Station    Chinese Restaurant                  Bank   
28           Bus Station    Chinese Restaurant                  Bank   
29           Bus Station    Chinese Restaurant                  Bank   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
21        Farmers Market     Currency Exchange          Deli / Bodega  
22        Farmers Market     Currency Exchange          Deli / Bodega  
23        Farmers Market     Currency Exchange          Deli / Bodega  
24        Farmers Market     Currency Exchange          Deli / Bodega  
26        Farmers Market     Currency Exchange          Deli / Bodega  
27        Farmers Market     Currency Exchange          Deli / Bodega  
28        Farmers Market     Currency Exchange          Deli / Bodega  
29        Farmers Market     Currency Exchange          Deli / Bodega

In [55]:
lake_county_merged.loc[lake_county_merged['Cluster Labels'] == 4, 
                     lake_county_merged.columns[[0,2] + list(range(5, lake_county_merged.shape[1]))]]

city postalcode  Cluster Labels 1st Most Common Venue  \
14  LEROY      46355               4             Laser Tag   

   2nd Most Common Venue 3rd Most Common Venue       4th Most Common Venue  \
14           Video Store        Discount Store  Construction & Landscaping   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
14     Convenience Store          Credit Union     Currency Exchange   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
14         Deli / Bodega                 Diner        Doctor's Office

In [56]:
# save as html file
# map_clusters.save('index.html')